# Creating metabolic models
This Jupyter Notebook uses carveme and cobra to create metabolic models of 7 bacterial species based on their protein fasta files and loads them into this notebook. Memote Reports are created to check model quality.

## Imports

In [1]:
import cobra
import os
import subprocess
from cobra.io import read_sbml_model, write_sbml_model
from contextlib import redirect_stdout, redirect_stderr
import pandas as pd
import ast

cobra.Configuration().solver = "cplex" # this was not doing anything that is why we set it manually below for that one model

# Carveme without constraints
At the beginning of my project, I used carveme like you see down below, i.e. without soft constraints. Later on we switched to using soft constraints as you can see in the second big section of this file.
Subprocess is used because carvme is a command line tool and subprocess uses the given command and executes on with the command line.

In [2]:
# Sequence_path id the location of the .faa files (protein fasta files) while models_path is location where metabolic models should be saved
sequence_path = "/home/lisa/Dokumente/Programmierung/Datasets"
models_path = "/home/lisa/Dokumente/Programmierung/Models/01_before_mass_balance"

In [3]:
# takes about 2min in total
# it does call the carve command multiple times in a row and does not use multiprocessing because I always get an error
# https://github.com/cdanielmachado/carveme/issues/99
# AA3 is gram-positive, rest is negative
for root in os.listdir(sequence_path):
    if ".faa" in root:
        root_name = root[:3]
        print("creating model for: ", root_name)
        if "AA3" in root:
            subprocess.run(["carve", sequence_path+"/"+root, "-u", "grampos", "--solver", "cplex", "--output", models_path+"/"+root_name+'.xml'])
        else:
            subprocess.run(["carve", sequence_path+"/"+root, "-u", "gramneg", "--solver", "cplex", "--output", models_path+"/"+root_name+'.xml'])
print("done")

creating model for:  AA4
creating model for:  AA7
creating model for:  AA2
creating model for:  AA5
creating model for:  AA6
creating model for:  AA3
creating model for:  AA1
done


## Load sbml models

In [3]:
model_list = [f"AA{i}.xml" for i in range(1, 8)]

In [4]:
# load carveme models
for i, model_name in enumerate(model_list, start=1):
    print("loading sbml model:", model_name)

    # next line suppresses the output of the read_sbml_model command; this is only done to have a nicer output
    # there are long outputs because exchange reactions are created and then ignored; if you wanna see the output just comment the "with open.." line
    with open(os.devnull, 'w') as f, redirect_stdout(f), redirect_stderr(f):
        model = read_sbml_model(f"{models_path}/{model_name}")

    model.solver = "cplex"
    globals()[f"AA{i}"] = model
print("done")

loading sbml model: AA1.xml
loading sbml model: AA2.xml
loading sbml model: AA3.xml
loading sbml model: AA4.xml
loading sbml model: AA5.xml
loading sbml model: AA6.xml
loading sbml model: AA7.xml
done


## Basic checks (solver, summary)

In [13]:
# check if solver is set to cplex
AA7.solver

In [33]:
# get flux through objective and upatee/secretion fluxes
AA3.summary()
# growth rate is way too high (72), max is around 3 (g Trockenmasse/Tag)
# daher mit memote checken wie Model so ist
# memote report snapshot file.xml

Metabolite,Reaction,Flux,C-Number,C-Flux
4abut_e,EX_4abut_e,74.63,4,1.65%
acnam_e,EX_acnam_e,6.342,11,0.39%
akg_e,EX_akg_e,988.3,5,27.39%
arg__L_e,EX_arg__L_e,625.4,6,20.80%
asn__L_e,EX_asn__L_e,10.47,4,0.23%
bz_e,EX_bz_e,0.004344,7,0.00%
ca2_e,EX_ca2_e,0.2261,0,0.00%
carn_e,EX_carn_e,0.02502,9,0.00%
cit_e,EX_cit_e,0.2261,6,0.01%
cl_e,EX_cl_e,0.2261,0,0.00%


## Memote Reports
I use subprocess again to redirect my code to the command line; however there would also be python version of memote that would require a bit more code

In [18]:
def memote_report(model_x, model_path_x, report_filename_x):
    print(f"Running memote for {model_x}")
    subprocess.run(["memote", "report", "snapshot", "--filename", report_filename_x, model_path_x, "--solver", "cplex"])

In [19]:
# Run memote for each model (~1-2min PER report)
for model in model_list:
    # print(f"Running memote for {model}")
    # path to one specific model
    model_path = f"{models_path}/{model}"
    # path where reports should be stored and report file name
    if "AA3" in model:
        report_filename = f"Reports/reports_grampos-neg/report_{model[:-4]}_grampos.html"
    else:
        report_filename = f"Reports/reports_grampos-neg/report_{model[:-4]}_gramneg.html"

    memote_report(model, model_path, report_filename)
print("done")

Running memote for AA1.xml


============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/lisa
plugins: anyio-4.9.0, typeguard-4.4.2
collected 146 items / 1 skipped

../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py . [  0%]
..FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...F....F.F...F.FF.FFFFFFFFFF..         [ 44%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 45%]
.....F........F.FFFFFF                                                   [ 60%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 60%]
........F                                                                [ 67%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_consistency.py F [ 67%]
FFFFFFF......s..FFFFFF.F.FFF                                             [ 86%]
../../miniconda3/envs/MA/lib/python3.10/site-packa

Writing snapshot report to 'Reports/reports_grampos-neg/report_AA1_gramneg.html'.


Running memote for AA2.xml


============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/lisa
plugins: anyio-4.9.0, typeguard-4.4.2
collected 146 items / 1 skipped

../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py . [  0%]
..FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...F....F.F...F.FF.FFFFFFFFFF..         [ 44%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 45%]
.....F........F..FFFFF                                                   [ 60%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 60%]
........F                                                                [ 67%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_consistency.py F [ 67%]
FFFFFFFFFFFFFsFFFFFFFF.F.FFF                                             [ 86%]
../../miniconda3/envs/MA/lib/python3.10/site-packa

...

Writing snapshot report to 'Reports/reports_grampos-neg/report_AA6_gramneg.html'.


Running memote for AA7.xml


============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/lisa
plugins: anyio-4.9.0, typeguard-4.4.2
collected 146 items / 1 skipped

../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py . [  0%]
..FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...F....F.F...F.FF.FFFFFFFFFF..         [ 44%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 45%]
.....F...........FFFFF                                                   [ 60%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 60%]
........F                                                                [ 67%]
../../miniconda3/envs/MA/lib/python3.10/site-packages/memote/suite/tests/test_consistency.py F [ 67%]
FF...........sFs.FFFFF.F.FFF                                             [ 86%]
../../miniconda3/envs/MA/lib/python3.10/site-packa

Writing snapshot report to 'Reports/reports_grampos-neg/report_AA7_gramneg.html'.


done


# CarveMe with soft constraints
https://carveme.readthedocs.io/en/latest/advanced.html#experimental-constraints
soft constraints allow us to exclude reactions from the model; this is helpful for us to exclude duplicate metabolites/reactions further down the pipeline; because of detailed manual curation efforts regarding charge balancing we identified model(s) that have reactions only coming from this model and creating duplicates in our models.
Currently, this is especially iYO844 (Bacillus subtilis model from 2007).

see 03_check_duplicates_and_loops Notebook; there i created the script to scrape the model info for every reaction in BIGG (rxn_model_origin) and then filtering reactions occurring only in a list of specified models (filtered_false_reactions)

In [2]:
rxn_model_origin = pd.read_csv("../reactions_bigg_model_origin.csv", quotechar='"')
rxn_model_origin["bigg_models"] = rxn_model_origin["bigg_models"].apply(ast.literal_eval)

In [8]:
rxn_model_origin

,bigg_id,name,bigg_models
0,11DOCRTSTRNtm,11-deoxycorticosterone intracellular transport,"[Recon3D, iMM1415, iCHOv1, RECON1, iCHOv1_DG44]"
1,10FTHF5GLUtl,"5-glutamyl-10FTHF transport, lysosomal","[Recon3D, iMM1415, iCHOv1, RECON1, iCHOv1_DG44]"
2,11DOCRTSTRNtr,11-deoxycorticosterone intracellular transport,"[Recon3D, iMM1415, iCHOv1, RECON1, iCHOv1_DG44]"
3,10FTHFtm,10-Formyltetrahydrofolate mitochondrial transp...,"[Recon3D, iMM1415, iCHOv1, RECON1, iCHOv1_DG44]"
4,10FTHF7GLUtl,"7-glutamyl-10FTHF transport, lysosomal","[Recon3D, iMM1415, iCHOv1, iAT_PLT_636, RECON1..."
...,...,...,...
28297,ZYMSTESTH_SCe,Zymosterol ester hydrolase yeast specific ex...,[iMM904]
28298,ZYMSTR,Zymosterol reductase,[iRC1080]
28299,ZYMSTt,Zymosterol reversible transport,"[iMM904, iND750]"
28300,Zn2tex,Zinc (Zn+2) transport via diffusion (extracell...,"[iECO103_1326, iE2348C_1286, iEC1344_C, iEC134..."


In [3]:
filtered_false_reactions = pd.read_csv("../filtered_false_reactions.tsv", quotechar='"')

In [7]:
filtered_false_reactions

,R_23CN2P1\t0
0,R_2PGLYCt6\t0
1,R_3AMBAt2\t0
2,R_3AMPt6\t0
3,R_3CMPt6\t0
4,R_3GMPt6\t0
...,...
314,R_UNK5\t0
315,R_UPP3MT_2\t0
316,R_X00002\t0
317,R_XYL14BASE\t0


## CarveMe

In [4]:
# Sequence_path id the location of the .faa files (protein fasta files) while models_path is location where metabolic models should be saved
sequence_path = "/home/lisa/Dokumente/Programmierung/Datasets"
models_path = "/home/lisa/Dokumente/Programmierung/Models/06_carveme"
constraints_path = "/home/lisa/Dokumente/Programmierung/Models/06_carveme/filtered_false_reactions.tsv"

carveme will create list of warnings for each model because not all reactions that we specified for constraints are in each model

In [5]:
for root in os.listdir(sequence_path):
    if ".faa" in root:
        root_name = root[:3]
        print("creating model for: ", root_name)
        if "AA3" in root:
            subprocess.run(["carve", sequence_path+"/"+root, "-u", "grampos", "--solver", "cplex", "--soft", constraints_path, "--output", models_path+"/"+root_name+'.xml'])
        else:
            subprocess.run(["carve", sequence_path+"/"+root, "-u", "gramneg", "--solver", "cplex", "--soft", constraints_path, "--output", models_path+"/"+root_name+'.xml'])
print("done")

creating model for:  AA4


/home/lisa/miniconda3/envs/MA/lib/python3.10/site-packages/carveme/reconstruction/carving.py:224: UserWarning: Soft constraints contain reactions not in the model:
R_PPTGS_BS
R_LAAL25
R_LAAL11
R_HP5CD_1
R_G1Dx
R_EX_lipt_e
R_EX_subtilisin_e
R_FACOAL150_anteiso
R_AHSERL3
R_DJENKabc
R_LIPO1S24_BS
R_ALCD19_L
R_DKII
R_FERXAabc
R_EX_ferrich_e
R_ALDD20x_1
R_ALDD15
R_PHYT3
R_IPDPUPT
R_LAAL9
R_LAAL28
R_LAAL12
R_AGPATr_BS
R_ALDD18
R_HEMEDS
R_LAAL39
R_DIPEPabc11
R_GALS4
R_AMMQT7
R_GALS2
R_LAAL35
R_EX_gly_gln__L_e
R_BIOMASS_BS_10
R_LAAL7
R_ARSt1
R_ECTOINEabc
R_EX_ala_L_Thr__L_e
R_UGT1_BS
R_CDPDSP_BS
R_DIPEPabc9
R_LIPTabc
R_UGT2_BS
R_CYOO3
R_PUNP8
R_LIPO2S24_BS
R_MLTG7
R_3AMBAt2
R_LYSLG_BS
R_ALDD8
R_DIPEPabc5
R_GALS1
R_EX_thiog_e
R_LAAL14
R_LAAL38
R_THIOGabc
R_EX_amylase_e
R_CBL2abc
R_EX_glycogen_e
R_LAAL10
R_DHNAOT7
R_DIPEPabc6
R_LAAL31
R_TMPt6
R_DIPEPabc7
R_TMPHi
R_DIPEPabc4
R_EX_arsna_e
R_EX_mops_e
R_ALDD7
R_G1PPDM
R_FERRICHabc
R_DKMD3M
R_CYOR3m
R_XYL14BASE
R_GBEZY
R_LAAL16
R_LAAL24
R_EX_djenk_e

creating model for:  AA7


/home/lisa/miniconda3/envs/MA/lib/python3.10/site-packages/carveme/reconstruction/carving.py:224: UserWarning: Soft constraints contain reactions not in the model:
R_TMPt6
R_DHNAOT7
R_GLYCOGENt
R_BILEt4
R_EX_bilea_e
R_BIOMASS_BS_10
R_EX_antim_e
R_LAAL18
R_FERXAabc
R_DKMD3M
R_FACOAL170_ISO
R_LAAL19
R_G3POA_BS
R_LAAL23
R_EX_djenk_e
R_ALDD9
R_FACOAL161_ISO
R_LAAL15
R_EX_arsni2_e
R_PROLBabc
R_LAAL29
R_UGT_BS
R_GBEZY
R_PYRIS
R_DKII
R_TECA3S45
R_CYOR3m
R_ALCD19_L
R_LAAL2
R_UGT1_BS
R_LAAL16
R_FACOAL171_anteiso
R_LIPO2S24_BS
R_EX_gly_gln__L_e
R_ANTIMt1
R_TECA1S45
R_LAAL20
R_EX_gly_met__L_e
R_LAAL33
R_HEMEDS
R_XYL14BASE
R_EX_amylase_e
R_LAAL38
R_FACOAL160_ISO
R_LAAL31
R_EX_pur_e
R_LAAL10
R_LIPO1S24_BS
R_DJENKabc
R_CYOO3
R_LAAL35
R_HXAD
R_MLTG7
R_HEMECS
R_ARSENBt2
R_FACOAL150_ISO
R_CDGPT_BS
R_LAAL8
R_LAAL5
R_LAAL32
R_GALS1
R_PSDC_BS
R_EX_ala_L_leu__L_e
R_PGPPH_BS
R_CBL2abc
R_LAAL3
R_EX_subtilisin_e
R_HP5CD_1
R_LAAL41
R_TECA4S_BS
R_ALDD20x_1
R_AHSERL3
R_LAAL21
R_EX_arsenb_e
R_LAAL26
R_ARSt1
R_PHY

creating model for:  AA2


...

creating model for:  AA3


/home/lisa/miniconda3/envs/MA/lib/python3.10/site-packages/carveme/reconstruction/carving.py:224: UserWarning: Soft constraints contain reactions not in the model:
R_LAAL26
R_ANTIMt1
R_DKMD3M
R_ECTOINEabc
R_LAAL39
R_PYRIS
R_EX_tmp_e
R_DIPEPabc2
R_ALDD14
R_GLYCOGENt
R_LAAL17
R_IPDPUPT
R_LAAL30
R_DHNAOT7
R_HEMEDS
R_BILEt4
R_ALDD15
R_LAAL42
R_PSDC_BS
R_LAAL20
R_ALDD20x_1
R_CYOR3m
R_EX_ala_L_leu__L_e
R_FERXAabc
R_LAAL33
R_EX_antim_e
R_FACOAL171_anteiso
R_EX_L_alagly_e
R_ARSENBt2
R_LAAL34
R_DIPEPabc11
R_LAAL21
R_LIPTabc
R_LAAL22
R_SPMDAT3
R_LAAL2
R_EX_arsna_e
R_DJENKabc
R_EX_lipt_e
R_EX_arsni2_e
R_PURabc1
R_LAAL35
R_DKII
R_LAAL16
R_DIPEPabc5
R_EX_thiog_e
R_CDGPT_BS
R_BIOMASS_BS_10
R_EX_glycogen_e
R_ALDD12
R_LAAL15
R_ALDD7
R_EX_amylase_e
R_EX_bilea_e
R_LAAL3
R_UGT1_BS
R_DIPEPabc6
R_LAAL7
R_AMMQT7
R_LAAL23
R_EX_gly_gln__L_e
R_PUNP8
R_LAAL19
R_PENCAC
R_LAAL27
R_CBL2abc
R_EX_pur_e
R_PGPPH_BS
R_LYSLG_BS
R_PHCYT_BS
R_LANTHabc
R_HP5CD_1
R_MOPSabc
R_LAAL13
R_G1PPDM
R_CYOO3
R_FACOAL161_ISO
R_EX_ferx

creating model for:  AA1


/home/lisa/miniconda3/envs/MA/lib/python3.10/site-packages/carveme/reconstruction/carving.py:224: UserWarning: Soft constraints contain reactions not in the model:
R_LAAL39
R_Q23DO
R_PURabc1
R_FERXAabc
R_ARSNAt1
R_TEICH45
R_ALDD8
R_PGPPH_BS
R_EX_ferrich_e
R_EX_ala_L_Thr__L_e
R_EX_amylase_e
R_AGPATr_BS
R_DKII
R_TMPt6
R_LAAL29
R_LAAL24
R_LAAL26
R_LANTHabc
R_LAAL19
R_ALDD13
R_HEMEDS
R_UGT_BS
R_LAAL17
R_LAAL6
R_EX_lanth_e
R_DIPEPabc9
R_THIOGabc
R_DIPEPabc6
R_G1PPDM
R_GALS1
R_FACOAL140_ISO
R_LAAL38
R_LAAL36
R_EX_ectoine_e
R_LAAL10
R_LIPO1S24_BS
R_ALDD7
R_EX_arsna_e
R_PYRIS
R_DIPEPabc7
R_LYSLG_BS
R_DKI
R_FACOAL150_anteiso
R_PHYT3
R_LAAL42
R_EX_ala_L_his__L_e
R_LAAL15
R_TECA3S45
R_LAAL14
R_LAAL31
R_LIPO2S24_BS
R_TECA1S45
R_MLTG7
R_FACOAL161_ISO
R_LAAL2
R_LAAL3
R_DIPEPabc4
R_DIPEPabc5
R_EX_ferxa_e
R_LAAL35
R_DIPEPabc11
R_LAAL12
R_FACOAL171_ISO
R_DIPEPabc2
R_LAAL32
R_LAAL25
R_EX_ala_L_leu__L_e
R_ALDD17
R_TECA2S45
R_AHSERL3
R_ALDD12
R_UGT1_BS
R_ARSENBabc
R_BLACT
R_LAAL4
R_EX_prolb_e
R_EX_gly_met

done


## Import models and save them
This is done to get a nice sbml version because memote was always annoyed and creating EX reactions on its own; now we do that in one go and while reading the models for the first time with read_sbml_model EX reactions are added.

In [6]:
# import models
models = {}
for model_name in (f for f in os.listdir(models_path) if f.endswith(".xml")):
    model = read_sbml_model(f"{models_path}/{model_name}")
    model.solver = "cplex"
    models[model_name[:3]] = model  # it takes first three letters of the xml filename as the model name

models = {key: models[key] for key in sorted(models.keys())}  # sorts the dictionary alphabetically (AA1...AA7) because of reasons it doesn't do this while creating

# this is specifically for my 7 models to be able to access them more easily; all models are stored in the "model" dict and can be accessed e.g. through models["AA1"] depending what you set as model name
AA1, AA2, AA3, AA4, AA5, AA6, AA7 = [models[f"AA{i}"] for i in range(1, 8)]

Adding exchange reaction EX_14glucan_e with default bounds for boundary metabolite: 14glucan_e.
Adding exchange reaction EX_25dkglcn_e with default bounds for boundary metabolite: 25dkglcn_e.
Adding exchange reaction EX_2m35mdntha_e with default bounds for boundary metabolite: 2m35mdntha_e.
Adding exchange reaction EX_2pglyc_e with default bounds for boundary metabolite: 2pglyc_e.
Adding exchange reaction EX_34dhbz_e with default bounds for boundary metabolite: 34dhbz_e.
Adding exchange reaction EX_35dnta_e with default bounds for boundary metabolite: 35dnta_e.
Adding exchange reaction EX_3mb_e with default bounds for boundary metabolite: 3mb_e.
Adding exchange reaction EX_3oxoadp_e with default bounds for boundary metabolite: 3oxoadp_e.
Adding exchange reaction EX_4abutn_e with default bounds for boundary metabolite: 4abutn_e.
Adding exchange reaction EX_4abz_e with default bounds for boundary metabolite: 4abz_e.
Adding exchange reaction EX_4hbald_e with default bounds for boundary me

In [7]:
for model_name, model in models.items():
    path = f"{models_path}/{model_name}.xml"
    write_sbml_model(model, path)